# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### To align table in markdown to left

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# Importing Cluster from cassandara.cluster module to connect to cassandra database
from cassandra.cluster import Cluster

# setting the cluster to 127.0.0.1
cluster = Cluster(["127.0.0.1"])

# Establishing the connection for queries execution
session = cluster.connect()

#### Create Keyspace

In [8]:
# Creating the Keyspace named sparkify 
try:
    session.execute("""
                CREATE KEYSPACE IF NOT EXISTS sparkify
                WITH REPLICATION =
                {'class': 'SimpleStrategy', 'replication_factor':1}
                
""")
except Error as e:
    print(e)

#### Set Keyspace

In [9]:
# Set KEYSPACE to sparkify
try:
    session.set_keyspace('sparkify')
except Error as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query1:
#### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
We need to model the following cql query to answer the above question:

    SELECT artist, song_title, song_length 
    FROM songs_session
    WERE sessionId = 338 AND itemInSession  = 4;

The table will be named as `songs_session`with a partition key and a clustering column . The structure of the table is as follows:

|Field|Type|Key|
|-----|----|----|
|session_id|INT|Partition|
|item_in_session|INT|Clustering|
|artist|TEXT||
|song_title|TEXT||
|song_length|TEXT||
|(session_id,item_id_session)||PRIMARY|

In [10]:
# query for creating songs_session table with the structure defined above.
query = """
        CREATE TABLE IF NOT EXISTS songs_session (
            session_id INT,
            item_in_session INT,
            artist TEXT,
            song_title TEXT,
            song_length FLOAT,
            PRIMARY KEY(session_id, item_in_session)
        );
"""
try:
    session.execute(query)
except Error as e:
    print(e)

#### Performing ETL

In [11]:
# The file from which the data is to be extracted
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # query for inserting into songs_session table
        query = "INSERT INTO songs_session(session_id, item_in_session, artist, song_title, song_length) "
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        
        # extracting the data from each line and inserting the data into songs_session table
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verifying the modelling and data insertion

In [12]:
# We'll execute the cql query to verify modelling and data insertion
query = """SELECT artist, song_title, song_length 
               FROM songs_session
               WHERE session_id = 338 AND item_in_session  = 4;
    """
try:
    # Creating pandas dataframe to view the data easily
    df = pd.DataFrame(list(session.execute(query)))
except Error as e:
    print(e)

# printing the dataframe to view the resutls
df

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2: 
#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

We need to model the following cql query to answer the above question:

    SELECT artist, song_title, user 
    FROM songs_user
    WHERE userId = 10 AND sessionId = 182;

The table will be named as `songs_user` with a composite partition key and a clustering column. The structure of the table is as follows:

|Field|Type|Key|
|-----|----|----|
|user_id|INT|Partition|
|session_id|INT|Partition|
|item_in_session|INT|Clustering|
|artist|TEXT||
|song_title|TEXT||
|user|TEXT||
|(user_id,session_id), item_id_session||PRIMARY|


In [16]:
# query for creating songs_user table with the structure defined above.
query = """
        CREATE TABLE IF NOT EXISTS songs_user (
            user_id INT,
            session_id INT,
            item_in_session INT,
            artist TEXT,
            song_title TEXT,
            user TEXT,
            PRIMARY KEY((user_id, session_id), item_in_session)
        );
"""
try:
    session.execute(query)
except Error as e:
    print(e)                    

#### Performing ETL

In [17]:
# The file from which the data is to be extracted
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # query for inserting into songs_user table
        query = "INSERT INTO songs_user(user_id, session_id, item_in_session, artist, song_title, user) "
        query = query + "VALUES(%s, %s, %s, %s, %s,%s)"
        
        # extracting the data from each line and inserting the data into songs_user table
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], (line[1] + ' ' + line[4])))

#### Verifying the modelling and data insertion

In [18]:
# We'll execute the cql query to verify modelling and data insertion
query = """SELECT artist, song_title, user 
               FROM songs_user
               WHERE user_id = 10 AND session_id = 182;
    """
try:
    # Creating pandas dataframe to view the data easily
    df = pd.DataFrame(list(session.execute(query)))
except Error as e:
    print(e)

# printing the dataframe to view the resutls
df

,artist,song_title,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


### Query 3: 
#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

We need to model the following cql query to answer the above question:

    SELECT user, song_title 
    FROM songs_history
    WHERE song_title = 'All Hands Against His Own' ;

The table will be named as `songs_history` with a partition key and a clustering column. The structure of the table is as follows:

|Field|Type|Key|
|-----|----|----|
|song_title|TEXT|Partition|
|user_id|INT|Clustering|
|user|TEXT||
|(song_title,user_id)||PRIMARY|


In [19]:
# query for creating songs_history table with the structure defined above.
query = """
        CREATE TABLE IF NOT EXISTS songs_history (
            song_title TEXT,
            user_id INT,
            user TEXT,
            PRIMARY KEY(song_title, user_id)
        );
"""
try:
    session.execute(query)
except Error as e:
    print(e)        

#### Performing ETL

In [25]:
# The file from which the data is to be extracted
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # query for inserting into songs_history table
        query = "INSERT INTO songs_history(song_title, user, user_id) "
        query = query + "VALUES(%s, %s, %s)"
        
        # extracting the data from each line and inserting the data into songs_history table
        session.execute(query, (line[9], (line[1] + ' ' + line[4]), int(line[10])))

#### Verifying the modelling and data insertion

In [26]:
# We'll execute the cql query to verify modelling and data insertion
query = """SELECT user, song_title 
               FROM songs_history
               WHERE song_title = 'All Hands Against His Own' ;
    """
try:
    # Creating pandas dataframe to view the data easily
    df = pd.DataFrame(list(session.execute(query)))
except Error as e:
    print(e)
    
# printing the dataframe to view the resutls
df

,user,song_title
0,Jacqueline Lynch,All Hands Against His Own
1,Tegan Levine,All Hands Against His Own
2,Sara Johnson,All Hands Against His Own


### Drop the tables before closing out the sessions

In [27]:
# Dropping all the tables using for loop and list
tables = ['songs_session', 'songs_user', 'songs_history']
for table in tables:
    try:
        session.execute("DROP TABLE IF EXISTS "+table)
    except Error as e:
        print(e)
    

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()